In [152]:
import requests 
from bs4 import BeautifulSoup
import datetime
import cPickle as pickle
names = []

#check date of the url to see if it was before 12/1/2014
#find all caption in the photo and store it into names
def getCaptions(u):
    url = u
    maxDate = datetime.datetime.strptime("December 1, 2014", "%B %d, %Y")
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    date = soup.find_all('div', class_ = 'panel-pane pane-node-created')
    if(len(date) > 0):
        d = date[0].text 
        #we only want the full date since it give us the day too
        pageDate = d[d.find(',') +1:].strip() 
        if datetime.datetime.strptime(pageDate, "%B %d, %Y") < maxDate :
            captions = soup.find_all('div', class_ = 'photocaption')
            for caption in captions:
                names.append(caption.text)

#at the moment there are 31 pages including starting page
#we get all the links from the first page
#then we loop through the rest of the 30 pages
url = "http://www.newyorksocialdiary.com/party-pictures"
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')
links = soup.find_all('span', class_ = 'views-field views-field-title')
for link in links:
    partyUrl = "http://www.newyorksocialdiary.com" + link.a.get('href')
    getCaptions(partyUrl)
for x in range(1,31):
    newUrl = url + "?page=" + str(x)
    response = requests.get(newUrl)
    soup = BeautifulSoup(response.text,'html.parser')
    links = soup.find_all('span', class_ = 'views-field views-field-title')
    for link in links:
        partyUrl = "http://www.newyorksocialdiary.com" + link.a.get('href')
        getCaptions(partyUrl)
pickle.dump(names,open("save.p","wb"))

In [58]:
import cPickle as pickle
captions = pickle.load( open("save.p","rb"))

def getNames(list):
    filtered = []
    for line in list:
    #assuming any captions under 250 characters are captions with people's names
        if len(line) < 250:
            #if statement for a list of people
            if "," in line or " and " in line:
                #if statement for 2 people and people with same last name
                if "," not in line:
                    temp = line.split(' ')
                    temp2 = line.split(" and ")
                    if temp[1] == "and" and len(temp)>3:
                        n = temp[0] + " " + temp[3]
                        filtered.append(n)
                        m = temp[2] + " " + temp[3]
                        filtered.append(m)
                    else:
                        filtered.append(temp2[0])
                        filtered.append(temp2[1])
                else:
                    #adds list of people into the filter list
                    temp = line.split(',')
                    for x in temp:
                        if " and " in x:
                            temp2 = x.split(" ")
                            #if statement for people with same last names in a list of people
                            if temp2[1] == "and" and temp2[0] != "" and len(temp2)>3:
                                n = temp2[0] + " " + temp2[3]
                                filtered.append(n)
                                m = temp2[2] + " " + temp2[3]
                                filtered.append(m) 
                            else:
                                filtered.append(x.replace(" and ", ""))
                        else:
                            filtered.append(x)

            else:
                #assuming people only have names with 4 words maximum including their titles
                #this is for captions with only 1 person in it
                temp = line.split(' ')
                if len(temp) <= 4:
                #check if first and last word is capitalized to see if it is a name or just words
                    if(temp[0].isupper and temp[len(temp)-1].isupper):
                        filtered.append(line)
                
    return filtered
#list of the titles that I found
titles = ["mayor","son","daughter","grandson","granddaughter","magician","chairman","chair","chairs","cice chairman",
         "vice chair", "chairmen","visitor","gala chairman","congressman","congresswoman","president",
         "chef","commissioner","actor","actors","actress","actresses","founding chairman","architech"
         "chief","ceo","wife","husband","fiance","fiancee"]
filter = []
for caption in captions:
    cap = caption.split()
    temp = [word for word in cap if word.lower() not in titles]
    filteredName = ' '.join(temp)
    filter.append(filteredName)
result = getNames(filter)
#converted to a set to remove duplicates
print len((set(result)))

92058
